In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [10]:
#graph adt


class Graph:
    
    """Graph ADT"""    
    def __init__(self):
        self.graph={}
        self.visited={}
            
    def append(self,vertexid,edge,weight):        
        """add/update new vertex,edge,weight"""        
        if vertexid not in self.graph.keys():      
            self.graph[vertexid]={}
            self.visited[vertexid]=0
        if edge not in self.graph.keys():      
            self.graph[edge]={}
            self.visited[edge]=0
        self.graph[vertexid][edge]=weight
        
    def reveal(self):
        """return adjacent list"""
        return self.graph
    
    def vertex(self):
        """return all vertices in the graph"""
        return list(self.graph.keys())
    
    def edge(self,vertexid):
        """return edge of a particular vertex"""
        return list(self.graph[vertexid].keys())
    
    def edge_reverse(self,vertexid):
        """return vertices directing to a particular vertex"""                
        return [i for i in self.graph if vertexid in self.graph[i]]
    
    def weight(self,vertexid,edge):
        """return weight of a particular vertex"""
        return (self.graph[vertexid][edge])
    
    def order(self):
        """return number of vertices"""
        return len(self.graph)
    
    def visit(self,vertexid):
        """visit a particular vertex"""
        self.visited[vertexid]=1
        
    def go(self,vertexid):
        """return the status of a particular vertex"""
        return self.visited[vertexid]
    
    def route(self):
        """return which vertices have been visited"""
        return self.visited
    
    def degree(self,vertexid):
        """return degree of a particular vertex"""
        return len(self.graph[vertexid])
    
    def mat(self):
        """return adjacent matrix"""        
        self.matrix=[[0 for _ in range(max(self.graph.keys())+1)] for i in range(max(self.graph.keys())+1)]        
        for i in self.graph:    
            for j in self.graph[i].keys():    
                self.matrix[i][j]=1        
        return self.matrix
    
    def remove(self,vertexid):  
        """remove a particular vertex and its underlying edges"""
        for i in self.graph[vertexid].keys():
            self.graph[i].pop(vertexid)
        self.graph.pop(vertexid)
        
    def disconnect(self,vertexid,edge):
        """remove a particular edge"""
        del self.graph[vertexid][edge]
        
    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
            
    def clear(self,vertexid=None,whole=False):
        """unvisit a particular vertex"""
        if whole:
            self.visited=dict(zip(self.graph.keys(),[0 for i in range(len(self.graph))]))
        elif vertexid:
            self.visited[vertexid]=0
        else:
            assert False,"arguments must satisfy whole=True or vertexid=int number"

In [11]:
#the structure of a* is similar to dijkstra's
#we compute distance from the beginning to the current vertex
#as well as the heuristic function
#for each vertex we iterate, we pick the minimum sum of both
#until we reach the destination
def a_star(ADT,start,end):
    """A* Algorithm"""
    #all weights in dcg must be positive 
    #otherwise we have to use bellman ford instead
    neg_check=[j for i in ADT.reveal() for j in ADT.reveal()[i].values()]
    assert min(neg_check)>=0,"negative weights are not allowed, please use Bellman-Ford"
    
    #queue in a* is the same as the queue in dijkstra
    queue={}
    queue[start]=0
    
    #same as dijkstra
    #distance keeps track of distance from starting vertex to any vertex
    distance={} 
    
    #heuristic keeps track of distance from ending vertex to any vertex
    heuristic={}
    
    #route is a dict of the summation of distance and heuristic
    route={}
    
    #criteria
    for i in ADT.vertex():
        
        #same as dijkstra
        distance[i]=float('inf')
        
        #manhattan distance
        heuristic[i]=abs(i[0]-end[0])+abs(i[1]-end[1])
        
    #same as dijkstra
    distance[start]=0

    #pred keeps track of how we get to the current vertex
    pred={}
    
    while queue:
        
        #same as dijkstra
        current=min(queue,key=queue.get)
        queue.pop(current)
        
        #find the minimum summation of both distances
        minimum=float('inf')

        for j in ADT.edge(current):
            
            #check if the current vertex can construct the optimal path
            #from the beginning and to the end
            distance[j]=distance[current]+ADT.weight(current,j)
            route[j]=distance[j]+heuristic[j]
            if route[j]<minimum:
                minimum=route[j]

        for j in ADT.edge(current):
            
            #we only append unvisited and unqueued vertices
            #note that we could have two vertices with the minimum summation
            #that is why we use a loop to find all valid vertices
            if (route[j]==minimum) and (ADT.go(j)==0) and (j not in queue):
                queue[j]=route[j]
                pred[j]=current
        
        #each vertex is visited only once
        ADT.visit(current)
        
        #traversal ends when the target is met
        if current==end:
            break        
    
    #create the shortest path by backtracking
    #trace the predecessor vertex from end to start
    previous=end
    path=[]
    while pred:
        path.insert(0, previous)
        if previous==start:
            break
        previous=pred[previous]
    
    #note that if we cant go from start to end
    #we may get inf for distance[end]
    #additionally, the path may not include start position
    return distance[end],path

In [12]:
# The main entry point for this module
def main():
        # Create a graph
        graph = Graph()
        
        # Create graph connections (Actual distance)
        graph.connect('Arad', 'Zerind', 75)
        graph.connect('Arad', 'Sibiiu', 140)
        graph.connect('Arad', 'Timisoara', 116)
        # Add Remaining Links From Example Given in Sides (Romania Map)
        graph.connect('Sibiiu', 'Fugaras', 99)
        graph.connect('Sibiiu', 'Rimnicu', 80)
        graph.connect('Rimnicu', 'Pitesti', 101)
        graph.connect('Fugaras', 'Bucharest', 211)
        graph.connect('Pitesti', 'Bucharest', 101)
        graph.connect('Giurgiu', 'Bucharest', 90)

        print(graph.get('Arad','Zerind'))

        # Make graph undirected, create symmetric connections
        graph.make_undirected()
        
        # Create heuristics (straight-line distance, air-travel distance) for Destination Bucharest
        heuristics = {}
        heuristics['Arad'] = 366
        heuristics['Zerind'] = 374
        heuristics['Sibiiu'] = 253
        heuristics['Timisoara'] = 329
        heuristics['Fugaras'] = 176
        heuristics['Rimnicu'] = 193
        heuristics['Pitesti'] = 10
        heuristics['Giurgiu'] = 77
        heuristics['Bucharest'] = 0
        
       
        print('\n')
        
        # Run search algorithm
        A_Star(graph, heuristics, 'Arad', 'Bucharest')
       
    # Tell python to run main method
if __name__ == "__main__": main()

AttributeError: 'Graph' object has no attribute 'graph_dict'